# Introduction

This project will help you generate the ics or google calendar. It seems that you will have to create an API if you want to use google calendar API generator. So I will firstly start with ical.

## import icalendar package

In [1]:
from icalendar import Calendar, Event
from datetime import datetime
from pytz import UTC # timezone
from pytz import timezone
from datetime import datetime, timedelta
import pickle

In [2]:
timezone('Europe/London')

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [6]:
# An example of read .ics file

# g = open("/Users/xihajun/Downloads/工作.ics",'rb')
g = open("../data/20220101.ics",'rb')
gcal = Calendar.from_ical(g.read())
events = []
for component in gcal.walk():
    if component.name == "VEVENT":
#         print(component.get('summary'))
#         print(component.get('dtstart').dt)
#         print(component.get('dtend').dt)
#         print(component.get('dtstamp').dt)
        event_start = component.get('dtstart').dt#.replace(tzinfo=timezone('Europe/London'))
        event_end = component.get('dtend').dt#.replace(tzinfo=timezone('Europe/London'))
        try:
            freq = component.get('RRULE').get('FREQ')[0]
        except:
            freq = 0
        if freq == 'DAILY':
            for i in range(0,20):
                events.append((event_start+timedelta(days=i),event_end+timedelta(days=i)))
        else:
            events.append((event_start,event_end))
g.close()

In [7]:
event_start

datetime.datetime(2022, 1, 1, 13, 30, tzinfo=<DstTzInfo 'Europe/London' GMT0:00:00 STD>)

In [8]:
with open("../data/component.pl","rb") as f:
    component = pickle.load(f)

In [18]:
# RRULE controls the freq
component.get('RRULE').get('FREQ')[0] == 'DAILY'

True

The task of this project would be finding some slots that are available for me given a pre-scheduled calendar. Within those slots, generate the time for me, given the courses I want to do and the number of hours I want to spend per week. (weekly initially, I might add functions to generate schedule for every two weeks or three weeks later)

In [10]:
import pickle
with open("../data/component.pl","wb") as f:
    pickle.dump(component,f)

In [46]:
# code from stackoverflow: https://stackoverflow.com/questions/10702224/python-algorithm-find-time-slots
from datetime import datetime, timedelta
from pytz import timezone


# appointments = [(datetime(2012, 5, 22, 10), datetime(2012, 5, 22, 10, 30)),
#                 (datetime(2012, 5, 22, 12), datetime(2012, 5, 22, 13)),
#                 (datetime(2012, 5, 22, 15, 30), datetime(2012, 5, 22, 17, 10))]

app = [(component.get('dtstart').dt,component.get('dtend').dt)]
start = datetime(2021, 2, 15, 9).replace(tzinfo=timezone('Europe/London'))
end = datetime(2021, 2, 15, 18).replace(tzinfo=timezone('Europe/London'))
hours = (start, end)

def get_slots(hours, events, duration=timedelta(hours=1)):
    
    # remove datatime.data events
    for i in events:
        # events[0][0] is datatime.datatime
        if type(i[0]) != type(events[0][0]):
            events.remove(i)
            
    today_event = [i for i in events if (i[0]>hours[0] and i[0]<hours[1])]
    slots = sorted([(hours[0], hours[0])] + today_event + [(hours[1], hours[1])])
    slots = sort_slots(slots)
#     print(slots)
    slots_obj = []
    
    for start, end in ((slots[i][1], slots[i+1][0]) for i in range(len(slots)-1)):
#         assert start <= end, "Cannot attend all appointments"
#         import pdb;pdb.set_trace()
        print(start + duration <= end)
        print(start + duration ,end)
        while start + duration <= end:
#             print("{:%H:%M} - {:%H:%M}".format(start, start + duration))
            slots_obj.append((start,start + duration))
            start += duration

    return slots_obj

In [47]:
def sort_slots(slots):
    for i in range(len(slots)-1):
        if slots[i][1]>slots[i+1][1]:
            end = slots[i+1][1]
            start = slots[i][0]
            slots[i+1] = (start,end)
        if slots[i+1][1]<slots[i][1]:
            end = slots[i][1]
            slots[i+1] = (start,end)
    return slots

In [63]:
start = datetime(2022, 1, 1, 9).replace(tzinfo=timezone('Europe/London'))
end = datetime(2022, 1, 1, 18).replace(tzinfo=timezone('Europe/London'))
hours = (start, end)

freetime = []
for i in range(0,6):
    hours = (start+timedelta(days=i), end+timedelta(days=i))
    free = get_slots(hours,events)
    freetime.append(free)

False
2022-01-01 10:00:00-00:01 2022-01-01 09:30:00+00:00
False
2022-01-01 11:30:00+00:00 2022-01-01 11:00:00+00:00
False
2022-01-01 13:00:00+00:00 2022-01-01 12:15:00+00:00
False
2022-01-01 14:15:00+00:00 2022-01-01 13:30:00+00:00
True
2022-01-01 15:30:00+00:00 2022-01-01 18:00:00-00:01
True
2022-01-02 10:00:00-00:01 2022-01-02 18:00:00-00:01
True
2022-01-03 10:00:00-00:01 2022-01-03 18:00:00-00:01
True
2022-01-04 10:00:00-00:01 2022-01-04 18:00:00-00:01
True
2022-01-05 10:00:00-00:01 2022-01-05 18:00:00-00:01
True
2022-01-06 10:00:00-00:01 2022-01-06 18:00:00-00:01


In [64]:
freetime_list = set([item for sublist in freetime for item in sublist])

In [65]:
print("I have "+str(len(freetime_list))+" hours free this weekday")

I have 48 hours free this weekday


In [66]:
import random
schedule = set(random.sample(freetime_list,10))
rest = freetime_list-schedule

In [67]:
[print(i[0].strftime("%m/%d/%Y, %H:%M:%S")) for i in schedule]

01/05/2022, 17:00:00
01/06/2022, 12:00:00
01/06/2022, 09:00:00
01/03/2022, 12:00:00
01/03/2022, 17:00:00
01/04/2022, 16:00:00
01/06/2022, 11:00:00
01/05/2022, 15:00:00
01/01/2022, 16:30:00
01/06/2022, 17:00:00


[None, None, None, None, None, None, None, None, None, None]

Yeah! It seems that my script works!! Happy happy joy joy!

Now I have to give how many hours I would like to spend during this week for given projects or courses.
(eg. cobra: 20 hours, FROG: 10 hours).

> python: which module do you want to do?
cobra
> python: how many hours do you want to spend on?
10
(25 hours left)
> continue or end (c/e)


In [68]:
projects = {}
while True:
    project_name = input("which module do you want to do?")
    project_hours = input("how many hours do you want to spend on?")
    schedule = set(random.sample(freetime_list,int(project_hours)))
    projects[project_name] = schedule
    freetime_list = freetime_list-schedule
    print("You have "+str(len(freetime_list))+" hours free this weekday")
    breakornot = input("continue or end (c/e)")
    if breakornot =='e':
        break

which module do you want to do?PCA
how many hours do you want to spend on?1
You have 47 hours free this weekday
continue or end (c/e)c
which module do you want to do?MLE
how many hours do you want to spend on?1
You have 46 hours free this weekday
continue or end (c/e)e


In [69]:
projects

{'PCA': {(datetime.datetime(2022, 1, 6, 9, 0, tzinfo=<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>),
   datetime.datetime(2022, 1, 6, 10, 0, tzinfo=<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>))},
 'MLE': {(datetime.datetime(2022, 1, 2, 14, 0, tzinfo=<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>),
   datetime.datetime(2022, 1, 2, 15, 0, tzinfo=<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>))}}

I would also like to generate emacs script:

```
**** TODO autoencoder
     SCHEDULED: <2021-02-15 Mon>
     - [X] def sequence2matrix
     - [ ] nerual network training
```
Here is the example of emacs script for todo and scheduling. I will just need to set the first two lines to show in emacs calendar.

# write to ics

In [70]:
from icalendar import Calendar, Event
from datetime import datetime
from pytz import UTC # timezone

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.dk//')
cal.add('version', '2.0')

k = 0
for name in projects.keys():
    for time in projects[name]:
        event = Event()
        event.add('summary', name)
        event.add('dtstart', time[0])
        event.add('dtend', time[1])
#         event.add('dtstamp', datetime(2005,4,4,0,10,0,tzinfo=UTC))
#         event_sk = start.strftime("%m/%d/%Y, %H:%M:%S") + str(k)
#         event['uid'] = hashlib.md5(event_sk.encode('utf-8')).hexdigest() + '@edu-ing.cn'
#         event.add('priority', 5)
        cal.add_component(event)

f = open('../data/20220101_TODO.ics', 'wb')
f.write(cal.to_ical())
f.close()